In [8]:
import vpython as vp

scene = vp.canvas(title='3D scene')

axis1 = vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(1,0,0), color=vp.vector(0,0,1))
axis2 = vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(0,1,0), color=vp.vector(0,1,0))
axis3 = vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(0,0,1), color=vp.vector(1,0,0))
circ = vp.shapes.circle(pos=(0,0), radius=0.2)
orbit = vp.paths.ellipse(width=5, height=3, up=vp.vector(0,1,1))
figure = vp.extrusion(path=orbit, shape=circ)

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
m = np.array([[-0.58375424, -0.74794831, -0.3159182 ],
 [-0.80877596,  0.50139873,  0.30737723],
 [ 0.07150129, -0.4349398,   0.89761625]])
rmt = np.linalg.inv(np.transpose(m))
scale = 1/(10**6)
scale2 = 1/(10**7)

def lambert_transfer(bd1, bd2, t0, T, l=0):
    #assume center of gravity (sun) at origin
    p1_ = bd1.state(t0)[:3]
    p2_ = bd2.state(t0+T)[:3]
    
    
    p1 = np.matmul(rmt, p1_)
    p2 = np.matmul(rmt, p2_)
    
    a = abs(np.linalg.norm(p2) - np.linalg.norm(p1))/2
    d = np.linalg.norm(p2 - p1) / 2
    b = (d**2 - a**2) ** 0.5
    
    f1 = np.array([0, 0, 0])
    x = a + l
    y = b * (x**2/a**2 - 1)**0.5
    s = (p2 - p1) / d / 2
    refs = np.cross(s, f1 - (p1+p2)/2)
    sn = np.array([s[1], -s[0], s[2]])
    refsn = np.cross(sn, f1 - (p1+p2)/2)
    if np.sign(refs[2]) != np.sign(refsn[2]):
        sn = np.array([-s[1], s[0], s[2]])
    f2 = s * x + sn * y + (p1 + p2)/2
    
    ae = (np.linalg.norm(p1) + np.linalg.norm(p1 - f2)) / 2
    c = np.linalg.norm(f2 - f1) / 2
    be = (ae**2 - c**2) ** 0.5
    GM = ss.body_list['SUN'].const('GM', 1)
    Te = timedelta(seconds=((4 * np.pi**2 * ae**3 / GM) ** 0.5)[0])
    
    def proj(v1, onto):
        return abs(np.sum(v1 * onto) / np.linalg.norm(onto))
    
    ctr = (f1 + f2) / 2
    r1 = p1 - ctr
    r2 = p2 - ctr
    cos1 = proj(r1, f2 - f1) / ae
    cos2 = proj(r2, f2 - f1) / ae
    th1 = np.arccos(cos1)
    th2 = np.arccos(cos2)
    th = np.pi - th1 - th2

    dt = th / (2*np.pi) * Te
    
     #drawing stuff
    """
    vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(3,0,0))
    vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(0,3,0))
    vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(0,0,3))
    vp.sphere(pos=vp.vector(*(p2_ * scale)), radius=1,  color=vp.vector(1,0,0))
    vp.sphere(pos=vp.vector(*(p1 * scale2)), radius=1,  color=vp.vector(0,0,1))
    vp.sphere(pos=vp.vector(*(p2 * scale2)), radius=1,  color=vp.vector(0,1,0))
    vp.sphere(pos=vp.vector(*(f2 * scale2)), radius=1,  color=vp.color.orange)
    
    circ = vp.shapes.circle(pos=(0,0), radius=0.2)
    path = vp.paths.ellipse(pos=vp.vector(*(ctr*scale2)),  
                             width=ae*scale2, height=be*scale2, up=vp.vector(0,0,1))
    orbit = vp.extrusion(path=path, shape=circ)
    orbit.rotate(angle = np.arctan(f2[1]/f2[0]), axis=vp.vector(0,0,1))
    """
  
    """
    org = lambda x: np.matmul(np.linalg.inv(rmt), x)
    vp.sphere(pos=vp.vector(*(p2_ * scale)), radius=1,  color=vp.vector(1,0,0))
    circ = vp.shapes.circle(pos=(0,0), radius=0.2)
    path = vp.paths.ellipse(pos=vp.vector(*org(ctr*scale)), 
                             width=ae*scale, height=be*scale) #+ vp.paths.rectangle(
        #width=10, height=10, up=vp.vector(*org(np.array([0,0,1]))))
    orbit = vp.extrusion(path=path, shape=circ)
        
    of2 = org(f2)
    vp.sphere(pos=vp.vector(*(of2*scale)), radius=1, color=vp.color.orange)
    vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(*(of2*scale)), shaftwidth=0.2)
    
    def rad_btwn(v1, v2):
        return np.arccos(np.sum(v1 * v2) / np.linalg.norm(v1) / np.linalg.norm(v2))
    
    vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(10, 0, 0), color=vp.vector(1,0,0))
    vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(0, 10, 0), color=vp.vector(0,1,0))
    vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(0, 0, 10), color=vp.vector(0,0,1))
    
    vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(*org(np.array([0, 0, 10]))))
    
    orbit.rotate(angle = rad_btwn(of2, np.array([1,0,0])), axis=vp.vector(0,0,1))
    norm = np.array([orbit.up.x, orbit.up.y, orbit.up.z])
    orbit.rotate(angle = rad_btwn(norm, org(np.array([0,0,1]))), axis=vp.vector(*of2))
    """
    
    #vp.box(pos=vp.vector(0,0,0), up=vp.vector(*org(np.array([0,0,1]))), length=200, width=200, height=0.1)
    
    
    #vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(*org(np.array((10, 0, 0)))))
    #vp.arrow(pos=vp.vector(0,0,0), axis=vp.vector(*org(np.array((0, 10, 0)))))
    
    #rad = rad_btwn(of2, np.array([-1, 0, 0]))
    #print(rad)
    #orbit.rotate(angle = -rad, axis=vp.vector(*org(np.array([0,0,1]))))
    

In [ ]:
vp.arrow(pos=V(0,0,0), axis=V(*(p1R*scale2)), color=V(1,0.5,0.5))
vp.arrow(pos=V(0,0,0), axis=V(*(p2R*scale2)), color=V(0.5,0.5,1))

circR = vp.shapes.circle(pos=(0,0), radius=0.2)
pathR = vp.paths.ellipse(pos=V(*(ctR*scale2)),  
    width=aeR*scale2, height=beR*scale2, up=V(0,0,1))

orbitR = vp.extrusion(path=pathR, shape=circR)
orbitR.rotate(angle = np.arctan(f2R[1]/f2R[0]), axis=V(0,0,1))

In [ ]:
"""

sin1 = (1 - cos1**2) **0.5
sin2 = (1 - cos2**2) **0.5
#velocity direction with ellipse's axes as basis
v1f = arr([sin1 * ae, -cos1 * be])
v2f = arr([sin2 * ae, cos2 * be])
v1f = v1f / norm(v1f)
v2f = v2f / norm(v2f)

ctRn = ctR / norm(ctR)
ref = np.cross(ctRn, p1R)
ctRp = arr([-ctRn[1], ctRn[0], ctRn[2]])
refc = np.cross(ctRn, ctRp)
#positive x points away from p1
if np.sign(ref[2]) == np.sign(refc[2]):
    ctRp = -ctRp
#convert back to regular basis in reduced R2 space
v1R = v1f[0] * ctRn + v1f[1] * ctRp 
v2R = v2f[0] * ctRn + v2f[1] * ctRp 
#convert back to regular basis in R3
v1n = Minv(v1R)
v2n = Minv(v2R)
"""